# Подключение библиотек:

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import torch
import torch.nn as nn

import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pymystem3 import Mystem
import string

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2023-06-02 20:26:44.827235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 20:26:45.925499: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-02 20:26:48.654476: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-02 20:26:48.654634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# Основной код:

### Предобработка данных:

#### С сайта:

In [3]:
dataFromWebsite = pd.DataFrame()
dataFromWebsite['Text'] = pd.Series()

In [4]:
with open("AUF.html", "r", encoding="utf-8") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")

def findElement(number):
    img_element = soup.find('img', class_='lazy lazy-hidden alignnone wp-image-' + str(number) + ' size-full')

    if img_element:
        parent_p_element = img_element.parent

        next_p_element = parent_p_element.find_next_sibling('p')

        if next_p_element:
            text = next_p_element.get_text()

            return text
        else:
            print('Next <p> element not found.')
    else:
        print('Image element not found.')

In [5]:
for k in range(2968, 3022):
    dataFromWebsite.loc[len(dataFromWebsite.index)] = [findElement(k)] 

Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.
Image element not found.


In [6]:
dataFromWebsite.head()

,Text
0,"Лучше быть последним — первым, чем первым — по..."
1,"На случай, если буду нужен, то я там же, где и..."
2,Если волк молчит то лучше его не перебивай.
3,"Каждый в цирке думает, что знает в цирке, но н..."
4,"Легко вставать, когда ты не ложился."


#### С другого сайта:

In [7]:
dataFromWebsites = pd.DataFrame()
dataFromWebsites['Text'] = pd.Series()

In [8]:
def parseAll(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    div_elements = soup.find_all('div', class_ = 'ads-color-box')
    
    if div_elements:
        contents = [div.text.strip() for div in div_elements]
        return contents
    
    return None

In [12]:
url1 = 'https://citatko.com/bez-rubriki/auf-tsitaty-pro-volkov'
dataFromWebsite1 = parseAll(url1)

url2 = 'https://citatko.com/temy/volk/tsitaty-pro-odinokogo-volka'
dataFromWebsite2 = parseAll(url2)

url3 = 'https://citatko.com/temy/volk/patsanskie-tsitaty-pro-volkov'
dataFromWebsite3 = parseAll(url3)

In [13]:
for content in dataFromWebsite1:
    dataFromWebsites.loc[len(dataFromWebsites.index)] = [content]

In [14]:
for content in dataFromWebsite2:
    dataFromWebsites.loc[len(dataFromWebsites.index)] = [content]

In [15]:
for content in dataFromWebsite3:
    dataFromWebsites.loc[len(dataFromWebsites.index)] = [content]

In [16]:
dataFromWebsites.head()

,Text
0,"Лучше быть последним – первым, чем первым – по..."
1,"Кем бы ты ни был, кем бы ты не стал, помни, гд..."
2,Если волк молчит то лучше его не перебивать.
3,"Делай как надо, как не надо не делай."
4,"Работа не волк, работа это ворк, а волк это хо..."


#### С файла:

In [17]:
dataFromFile = pd.read_fwf('slogans.txt')

In [18]:
dataFromFile.head()

,Text
0,Я знал настоящего волка — он был просрочен и к...
1,Волк может отгрызть себе жопу чтобы не стать п...
2,"Хочешь, будь волком. Это твоё дело. Главное на..."
3,"В этой жизни ты либо волк, либо не волк."
4,"Если волк молчит, то лучше его не перебивай."


In [19]:
data = pd.concat([dataFromWebsite, dataFromFile, dataFromWebsites], ignore_index = True, sort = False)
data.dropna(inplace = True)
data.reset_index(inplace = True, drop = True)

In [20]:
data

,Text
0,"Лучше быть последним — первым, чем первым — по..."
1,"На случай, если буду нужен, то я там же, где и..."
2,Если волк молчит то лучше его не перебивай.
3,"Каждый в цирке думает, что знает в цирке, но н..."
4,"Легко вставать, когда ты не ложился."
...,...
227,"Главное не образ льва, а дух волка. И волки сы..."
228,"Волк на холме не так голоден, как волк, взбира..."
229,"Щедрому волку не жалко травы для барана, а жад..."
230,Волки не ждут предательства от своих в стае — ...


In [21]:
m = Mystem()

def preprocessText(text):
    whitelist = set('ёйцукенгшщзхъфывапролджэячсмитьбю ЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ')
    text = ''.join(filter(whitelist.__contains__, text))
    text = text.lower()
    text = ''.join(m.lemmatize(text)).strip()
    
    return text

In [22]:
data['prepText'] = data['Text'].apply(preprocessText)

In [23]:
data

,Text,prepText
0,"Лучше быть последним — первым, чем первым — по...",хорошо быть последний первый чем первый посл...
1,"На случай, если буду нужен, то я там же, где и...",на случай если быть нужный то я там же где и б...
2,Если волк молчит то лучше его не перебивай.,если волк молчать то хорошо он не перебивать
3,"Каждый в цирке думает, что знает в цирке, но н...",каждый в цирк думать что знать в цирк но не ка...
4,"Легко вставать, когда ты не ложился.",легко вставать когда ты не ложиться
...,...,...
227,"Главное не образ льва, а дух волка. И волки сы...",главное не образ лев а дух волк и волк сытый и...
228,"Волк на холме не так голоден, как волк, взбира...",волк на холм не так голодный как волк взбирать...
229,"Щедрому волку не жалко травы для барана, а жад...",щедрый волк не жалко трава для баран а жадный ...
230,Волки не ждут предательства от своих в стае — ...,волк не ждать предательство от свой в стая он...


In [32]:
tokenizer = Tokenizer(char_level = False)

tokenizer.fit_on_texts(data['Text'])

sequences = tokenizer.texts_to_sequences(data['Text'])

sequences = pad_sequences(sequences, padding = 'post')

least_freq_word = min(tokenizer.word_index, key = tokenizer.word_index.get)

print(sequences[0])

[ 9 14 24 28  3  1 21 20  4 17  1 18  2 11  9  3 13  6  7 15  1 33  1 18
  3 12  8 20 15 16  1 24  3 15  1 18  3 12  8 20 15  1 33  1 18  2 11  9
  3 13  6  7 15 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


### Объявление нейронки:

In [33]:
class SloganGenerator(nn.Module):
    def __init__(self, vocabularySize, embeddingDim, hiddenDim, numLayers):
        super(SloganGenerator, self).__init__()
        self.embedding = nn.Embedding(vocabularySize, embeddingDim)
        self.lstm = nn.LSTM(embeddingDim, hiddenDim, numLayers, batch_first = True)
        self.fc = nn.Linear(hiddenDim, vocabularySize)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

### Инициализация модели:

In [34]:
vocabularySize = len(tokenizer.word_index) + 1
embeddingDim = 256
hiddenDim = 128
numLayers = 2

model = SloganGenerator(vocabularySize, embeddingDim, hiddenDim, numLayers)
print(model)

SloganGenerator(
  (embedding): Embedding(49, 256)
  (lstm): LSTM(256, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=49, bias=True)
)


### Обучение модели:

In [36]:
model = model.to(device)

sequencesTensor = torch.tensor(sequences).long()

inputSequences = sequencesTensor[:, :-1].to(device)
targetSequences = sequencesTensor[:, 1:].to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

numEpochs = 300 # Подобрать
for epoch in range(numEpochs):
    model.train()
    optimizer.zero_grad()
    output = model(inputSequences)

    loss = criterion(output.transpose(1, 2), targetSequences)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f'Epoch {epoch + 1}/{numEpochs}, Loss: {loss.item()}')

Epoch 20/300, Loss: 0.7429830431938171


KeyboardInterrupt: 

# Генерируем слоганы:

In [28]:
def generateSlogan(model, tokenizer, seedText, minWordLength = 10, maxWordLength = 12, temperature = 2):
    model.eval()

    seed_words = seedText.lower().split()

    input_sequence = [tokenizer.word_index.get(word, tokenizer.word_index[least_freq_word]) for word in seed_words]
    slogan = ""

    while len(slogan.split()) < minWordLength or len(slogan.split()) > maxWordLength:
        input_sequence_tensor = torch.tensor([input_sequence]).long().to(device)
        output = model(input_sequence_tensor)

        output_probs = torch.softmax(output / temperature, dim = -1)

        next_token = torch.multinomial(output_probs[:, -1], num_samples = 1).item()

        input_sequence.append(next_token)

        slogan = ' '.join(tokenizer.index_word.get(token, '') for token in input_sequence[1:])

    slogan = ' '.join(slogan.split()[:maxWordLength])

    return f'{seedText} - {slogan}'

In [24]:
print(generateSlogan(model, tokenizer, 'Данила'))

Данила - верь тому кому не веришь ведь вера в не веру


In [29]:
print(generateSlogan(model, tokenizer, 'Волк'))

Волк - – единственный из зверей который может пойти в бой на более сильного


In [28]:
print(generateSlogan(model, tokenizer, 'Волк'))

Волк - никогда не легко настоящих не укусил но и своих приручить


In [39]:
print(generateSlogan(model, tokenizer, 'Саша')) # temperature = 3

Саша - поставит внутри в санитар может меня думает уперты волчья легенда


In [40]:
print(generateSlogan(model, tokenizer, 'Максим'))

Максим - ударов тот на слабее забывают вырос волку оступиться… видит жадному


In [45]:
print(generateSlogan(model, tokenizer, 'Артём'))

Артём - душит сыр со раньше нов свое зато удивленно встал волчица


In [50]:
print(generateSlogan(model, tokenizer, 'Саня'))

Саня - смотрите что бьют не так уже довольствуется вашей маской человеку


# Бонус: генерируем картинки

In [29]:
import cv2
from skimage.transform import swirl
from PIL import Image
import numpy as np

import random
from datetime import datetime
import json

In [30]:
json_file = 'images.json'

def chooseRandomElement():
    with open(json_file, 'r') as file:
        JSONData = json.load(file)

    random.seed(random.seed(datetime.now().timestamp()))

    randomElement = random.choice(JSONData)
    return randomElement

def putTextOnImage(image, topLeft, width, height, text):
    overlay = np.zeros_like(image, dtype=np.uint8)

    fontFace = cv2.FONT_HERSHEY_COMPLEX
    fontColor = (255, 255, 255)

    # bottomRight = (topLeft[0] + width, topLeft[1] + height)
    # cv2.rectangle(overlay, topLeft, bottomRight, (0, 0, 255), -1)

    # Try different font sizes until the text fits inside the rectangle
    fontScale = 1
    while True:
        lines = []
        line = ''
        words = text.split()
        for word in words:
            line_test = line + ' ' + word if line else word
            text_size, _ = cv2.getTextSize(line_test, fontFace, fontScale, 1)
            if text_size[0] <= width:
                line = line_test
            else:
                lines.append(line)
                line = word
        lines.append(line)

        line_height = text_size[1]
        if line_height * len(lines) <= height:
            break
        fontScale -= 0.3

    text_org = (topLeft[0] + int((width - width) / 2), topLeft[1] + int((height - line_height * len(lines)) / 2))
    for line in lines:
        cv2.putText(overlay, line, text_org, fontFace, fontScale, fontColor, 1, cv2.LINE_4)
        text_org = (text_org[0], text_org[1] + line_height)

    result = cv2.addWeighted(image, 1, overlay, 0.5, 0)

    return result

def generatePicture(model, tokenizer, seedText, generateBool = True):
    if generateBool:
        text = generateSlogan(model, tokenizer, seedText)
    else:
        text = seedText

    randomImage = chooseRandomElement()

    img = cv2.imread(randomImage['path_to_picture'])

    Cx = randomImage['coordinates']['x']
    Cy = randomImage['coordinates']['y']

    maxAmount = 3
    radius = randomImage['radius']
    angle = 0
    numFrames = 50
    delay = 1

    Tx = randomImage['text_coordinates']['x']
    Ty = randomImage['text_coordinates']['y']

    Tw = randomImage['text_coordinates']['w']
    Th = randomImage['text_coordinates']['h']

    frames = []
    # Loop and increase swirl
    for i in range(0, numFrames):
        amount = i * maxAmount / numFrames

        result = swirl(img, center = (Cx,Cy), rotation = angle, strength = amount, radius = radius, preserve_range = True).astype(np.uint8)

        result = putTextOnImage(result, (Tx, Ty), Tw, Th, text)

        cv2.waitKey(delay)

        result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        pilResult = Image.fromarray(result)
        frames.append(pilResult)

    frames[0].save(str(seedText) + '.gif',save_all = True, append_images = frames[1:], optimize = False, duration = delay, loop = 0)

In [51]:
generatePicture(model, tokenizer, 'Артем')

In [31]:
generatePicture(model, tokenizer, 'Пенис')

In [53]:
generatePicture(model, tokenizer, 'Данила')